In [ ]:
!pip install bertviz

In [ ]:
# Import all necessary libraries and functions
import torch
from bertviz import head_view, model_view, neuron_view
from transformers import BertTokenizer, BertModel
from bertviz.transformers_neuron_view import BertModel as NeuronViewModel
from bertviz.transformers_neuron_view import BertTokenizer as NeuronViewTokenizer

In [ ]:
# --- 1. Load All Models and Tokenizers ---
print("Loading models... This may take a moment.")
model_version = 'bert-base-uncased'
model_type = 'bert'

# Models for head_view and model_view
model_hv = BertModel.from_pretrained(model_version, output_attentions=True)
tokenizer_hv = BertTokenizer.from_pretrained(model_version)
model_hv.eval()

# Models for neuron_view (requires special wrappers)
model_nv = NeuronViewModel.from_pretrained(model_version, output_attentions=True)
tokenizer_nv = NeuronViewTokenizer.from_pretrained(model_version)
print("Models loaded successfully.")

# --- 2. Create a Single, Reusable Visualization Function ---

@torch.no_grad()
def visualize_all_views(sentence_a, sentence_b=None, nv_layer=2, nv_head=8):
    """
    Generates head, model, and neuron views for one or two sentences.

    Args:
        sentence_a (str): The first sentence.
        sentence_b (str, optional): The second sentence. Defaults to None.
        nv_layer (int): The layer to display in neuron_view.
        nv_head (int): The head to display in neuron_view.
    """

    # --- A. Prepare inputs for head_view and model_view ---
    if sentence_b:
        print("\nPreparing views for TWO sentences...")
        inputs = tokenizer_hv.encode_plus(sentence_a, sentence_b, return_tensors='pt')
        token_type_ids = inputs['token_type_ids']
        sentence_b_start = token_type_ids[0].tolist().index(1)
    else:
        print("\nPreparing views for a SINGLE sentence...")
        inputs = tokenizer_hv.encode_plus(sentence_a, return_tensors='pt')
        token_type_ids = None
        sentence_b_start = None

    input_ids = inputs['input_ids']
    tokens = tokenizer_hv.convert_ids_to_tokens(input_ids[0])
    attention = model_hv(input_ids, token_type_ids=token_type_ids)[-1]

    # --- B. Display all three visualizations ---
    print("\n--- Head View ---")
    head_view(attention, tokens, sentence_b_start=sentence_b_start)

    print("\n--- Model View ---")
    model_view(attention, tokens, sentence_b_start=sentence_b_start)

    print(f"\n--- Neuron View (Layer: {nv_layer}, Head: {nv_head}) ---")
    neuron_view.show(model_nv, model_type, tokenizer_nv, sentence_a, sentence_b, layer=nv_layer, head=nv_head)


In [ ]:
# Scenario 1: Analyze a single, ambiguous sentence
single_sentence = "The man saw a boy with a telescope"
visualize_all_views(sentence_a=single_sentence)

In [ ]:
# Scenario 2: Analyze how context from a second sentence resolves ambiguity
sentence_1 = "The man saw a boy with a telescope"
sentence_2 = "He adjusted the focus for a clearer view"
visualize_all_views(sentence_a=sentence_1, sentence_b=sentence_2, nv_layer=8, nv_head=6)